In [3]:
import numpy as np
from scipy.stats import pearsonr
from scipy import ndimage
import matplotlib.pyplot as plt

def psnr(original, reconstructed, data_range=1.0):
    """Calculate Peak Signal-to-Noise Ratio"""
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(data_range / np.sqrt(mse))


def ssim(img1, img2, data_range=1.0, win_size=11):
    """
    Calculate Structural Similarity Index
    Simplified implementation without scikit-image dependency
    """
    C1 = (0.01 * data_range) ** 2
    C2 = (0.03 * data_range) ** 2
    
    # Create Gaussian window
    sigma = 1.5
    truncate = 3.5
    radius = int(truncate * sigma + 0.5)
    
    # Simple averaging window as fallback
    window = np.ones((win_size, win_size)) / (win_size ** 2)
    
    mu1 = ndimage.convolve(img1, window, mode='reflect')
    mu2 = ndimage.convolve(img2, window, mode='reflect')
    
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2
    
    sigma1_sq = ndimage.convolve(img1 ** 2, window, mode='reflect') - mu1_sq
    sigma2_sq = ndimage.convolve(img2 ** 2, window, mode='reflect') - mu2_sq
    sigma12 = ndimage.convolve(img1 * img2, window, mode='reflect') - mu1_mu2
    
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / \
               ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    
    return np.mean(ssim_map)


def comprehensive_deconvolution_analysis(original, blurry, deconvolved, psf=None):
    """
    Complete performance analysis for deconvolution with visualization.
    Works without scikit-image dependency.
    
    Parameters:
    -----------
    original : numpy array
        Original sharp image (ground truth)
    blurry : numpy array
        Simulated blurry image
    deconvolved : numpy array
        Richardson-Lucy deconvolved result
    psf : numpy array, optional
        Point spread function used
    """
    
    # Normalize images
    def normalize(img):
        img_min, img_max = img.min(), img.max()
        if img_max == img_min:
            return np.zeros_like(img)
        return (img - img_min) / (img_max - img_min)
    
    orig_norm = normalize(original)
    blur_norm = normalize(blurry)
    deconv_norm = normalize(deconvolved)
    
    metrics = {}
    
    # === Quality Metrics ===
    # PSNR
    metrics['psnr_blurry'] = psnr(orig_norm, blur_norm, data_range=1.0)
    metrics['psnr_deconvolved'] = psnr(orig_norm, deconv_norm, data_range=1.0)
    metrics['psnr_gain'] = metrics['psnr_deconvolved'] - metrics['psnr_blurry']
    
    # SSIM
    metrics['ssim_blurry'] = ssim(orig_norm, blur_norm, data_range=1.0)
    metrics['ssim_deconvolved'] = ssim(orig_norm, deconv_norm, data_range=1.0)
    metrics['ssim_gain'] = metrics['ssim_deconvolved'] - metrics['ssim_blurry']
    
    # MSE
    metrics['mse_blurry'] = np.mean((orig_norm - blur_norm) ** 2)
    metrics['mse_deconvolved'] = np.mean((orig_norm - deconv_norm) ** 2)
    metrics['mse_reduction_pct'] = (metrics['mse_blurry'] - metrics['mse_deconvolved']) / metrics['mse_blurry'] * 100
    
    # MAE (Mean Absolute Error)
    metrics['mae_blurry'] = np.mean(np.abs(orig_norm - blur_norm))
    metrics['mae_deconvolved'] = np.mean(np.abs(orig_norm - deconv_norm))
    
    # Correlation
    metrics['corr_blurry'] = pearsonr(orig_norm.flatten(), blur_norm.flatten())[0]
    metrics['corr_deconvolved'] = pearsonr(orig_norm.flatten(), deconv_norm.flatten())[0]
    
    # === Sharpness Metrics ===
    def laplacian_variance(img):
        laplacian = ndimage.laplace(img)
        return laplacian.var()
    
    metrics['sharpness_original'] = laplacian_variance(orig_norm)
    metrics['sharpness_blurry'] = laplacian_variance(blur_norm)
    metrics['sharpness_deconvolved'] = laplacian_variance(deconv_norm)
    metrics['sharpness_recovery_pct'] = (metrics['sharpness_deconvolved'] / metrics['sharpness_original']) * 100
    
    # Edge strength
    def edge_strength(img):
        gy, gx = np.gradient(img)
        return np.mean(np.sqrt(gx**2 + gy**2))
    
    metrics['edge_original'] = edge_strength(orig_norm)
    metrics['edge_blurry'] = edge_strength(blur_norm)
    metrics['edge_deconvolved'] = edge_strength(deconv_norm)
    metrics['edge_recovery_pct'] = (metrics['edge_deconvolved'] / metrics['edge_original']) * 100
    
    # === Residual Analysis ===
    metrics['residual_blurry_mean'] = np.mean(np.abs(orig_norm - blur_norm))
    metrics['residual_deconvolved_mean'] = np.mean(np.abs(orig_norm - deconv_norm))
    metrics['residual_blurry_std'] = np.std(orig_norm - blur_norm)
    metrics['residual_deconvolved_std'] = np.std(orig_norm - deconv_norm)
    
    # Print results
    print("=" * 70)
    print("RICHARDSON-LUCY DECONVOLUTION PERFORMANCE ANALYSIS")
    print("=" * 70)
    
    print("\n📊 IMAGE QUALITY METRICS")
    print("-" * 70)
    print(f"{'Metric':<25} {'Blurry':<15} {'Deconvolved':<15} {'Gain/Change':<15}")
    print("-" * 70)
    print(f"{'PSNR (dB)':<25} {metrics['psnr_blurry']:>14.2f} {metrics['psnr_deconvolved']:>14.2f} {metrics['psnr_gain']:>+14.2f}")
    print(f"{'SSIM':<25} {metrics['ssim_blurry']:>14.4f} {metrics['ssim_deconvolved']:>14.4f} {metrics['ssim_gain']:>+14.4f}")
    print(f"{'MSE':<25} {metrics['mse_blurry']:>14.6f} {metrics['mse_deconvolved']:>14.6f} {metrics['mse_reduction_pct']:>+13.2f}%")
    print(f"{'MAE':<25} {metrics['mae_blurry']:>14.6f} {metrics['mae_deconvolved']:>14.6f}")
    print(f"{'Correlation':<25} {metrics['corr_blurry']:>14.4f} {metrics['corr_deconvolved']:>14.4f}")
    
    print("\n🔍 SHARPNESS & EDGE METRICS")
    print("-" * 70)
    print(f"{'Laplacian Variance':<25} {metrics['sharpness_blurry']:>14.6f} {metrics['sharpness_deconvolved']:>14.6f} ({metrics['sharpness_recovery_pct']:>6.1f}% of orig)")
    print(f"{'Edge Strength':<25} {metrics['edge_blurry']:>14.6f} {metrics['edge_deconvolved']:>14.6f} ({metrics['edge_recovery_pct']:>6.1f}% of orig)")
    print(f"{'Original Sharpness':<25} {metrics['sharpness_original']:>14.6f}")
    print(f"{'Original Edge Strength':<25} {metrics['edge_original']:>14.6f}")
    
    print("\n📉 RESIDUAL ANALYSIS")
    print("-" * 70)
    print(f"{'Residual Mean':<25} {metrics['residual_blurry_mean']:>14.6f} {metrics['residual_deconvolved_mean']:>14.6f}")
    print(f"{'Residual Std Dev':<25} {metrics['residual_blurry_std']:>14.6f} {metrics['residual_deconvolved_std']:>14.6f}")
    
    print("\n✨ SUMMARY")
    print("-" * 70)
    if metrics['psnr_gain'] > 3:
        quality = "Excellent"
    elif metrics['psnr_gain'] > 1:
        quality = "Good"
    elif metrics['psnr_gain'] > 0:
        quality = "Moderate"
    else:
        quality = "Poor (no improvement)"
    print(f"Overall Quality Improvement: {quality}")
    print(f"PSNR Gain: {metrics['psnr_gain']:.2f} dB")
    print(f"SSIM Gain: {metrics['ssim_gain']:.4f}")
    print(f"MSE Reduction: {metrics['mse_reduction_pct']:.1f}%")
    print(f"Sharpness Recovery: {metrics['sharpness_recovery_pct']:.1f}%")
    print("=" * 70)
    
    # Visualization
    create_performance_visualization(original, blurry, deconvolved, metrics, psf)
    
    return metrics


def create_performance_visualization(original, blurry, deconvolved, metrics, psf=None):
    """Create comprehensive visualization of results"""
    
    def scale_image(img, percentile=99.5):
        vmin = np.percentile(img, 100 - percentile)
        vmax = np.percentile(img, percentile)
        return vmin, vmax
    
    def normalize(img):
        img_min, img_max = img.min(), img.max()
        if img_max == img_min:
            return np.zeros_like(img)
        return (img - img_min) / (img_max - img_min)
    
    orig_norm = normalize(original)
    blur_norm = normalize(blurry)
    deconv_norm = normalize(deconvolved)
    
    # Create figure
    if psf is not None:
        fig = plt.figure(figsize=(20, 12))
        gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    else:
        fig = plt.figure(figsize=(20, 10))
        gs = fig.add_gridspec(2, 4, hspace=0.3, wspace=0.3)
    
    # Row 1: Images
    ax1 = fig.add_subplot(gs[0, 0])
    vmin, vmax = scale_image(original)
    ax1.imshow(original, cmap='gray', vmin=vmin, vmax=vmax)
    ax1.set_title("Original (Ground Truth)", fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    ax2 = fig.add_subplot(gs[0, 1])
    vmin, vmax = scale_image(blurry)
    ax2.imshow(blurry, cmap='gray', vmin=vmin, vmax=vmax)
    ax2.set_title(f"Blurry\nPSNR: {metrics['psnr_blurry']:.2f} dB", fontsize=12)
    ax2.axis('off')
    
    ax3 = fig.add_subplot(gs[0, 2])
    vmin, vmax = scale_image(deconvolved)
    ax3.imshow(deconvolved, cmap='gray', vmin=vmin, vmax=vmax)
    ax3.set_title(f"Deconvolved (RL)\nPSNR: {metrics['psnr_deconvolved']:.2f} dB", fontsize=12)
    ax3.axis('off')
    
    # Error maps
    ax4 = fig.add_subplot(gs[0, 3])
    error_map = np.abs(orig_norm - deconv_norm)
    im = ax4.imshow(error_map, cmap='hot')
    ax4.set_title(f"Absolute Error Map\nMean: {metrics['residual_deconvolved_mean']:.4f}", fontsize=12)
    ax4.axis('off')
    plt.colorbar(im, ax=ax4, fraction=0.046)
    
    # Row 2: Difference images
    ax5 = fig.add_subplot(gs[1, 0])
    diff_blurry = orig_norm - blur_norm
    ax5.imshow(diff_blurry, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
    ax5.set_title("Difference: Original - Blurry", fontsize=12)
    ax5.axis('off')
    
    ax6 = fig.add_subplot(gs[1, 1])
    diff_deconv = orig_norm - deconv_norm
    ax6.imshow(diff_deconv, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
    ax6.set_title("Difference: Original - Deconvolved", fontsize=12)
    ax6.axis('off')
    
    # Metrics comparison
    ax7 = fig.add_subplot(gs[1, 2])
    categories = ['PSNR\n(normalized)', 'SSIM', 'Correlation']
    blurry_vals = [
        metrics['psnr_blurry'] / 50,  # Normalize to ~0-1
        metrics['ssim_blurry'],
        metrics['corr_blurry']
    ]
    deconv_vals = [
        metrics['psnr_deconvolved'] / 50,
        metrics['ssim_deconvolved'],
        metrics['corr_deconvolved']
    ]
    
    x = np.arange(len(categories))
    width = 0.35
    ax7.bar(x - width/2, blurry_vals, width, label='Blurry', alpha=0.8)
    ax7.bar(x + width/2, deconv_vals, width, label='Deconvolved', alpha=0.8)
    ax7.set_ylabel('Score', fontsize=10)
    ax7.set_title('Quality Metrics Comparison', fontsize=12, fontweight='bold')
    ax7.set_xticks(x)
    ax7.set_xticklabels(categories, fontsize=9)
    ax7.legend()
    ax7.grid(axis='y', alpha=0.3)
    ax7.set_ylim([0, 1.1])
    
    # Histogram comparison
    ax8 = fig.add_subplot(gs[1, 3])
    ax8.hist(orig_norm.flatten(), bins=50, alpha=0.5, label='Original', density=True)
    ax8.hist(blur_norm.flatten(), bins=50, alpha=0.5, label='Blurry', density=True)
    ax8.hist(deconv_norm.flatten(), bins=50, alpha=0.5, label='Deconvolved', density=True)
    ax8.set_xlabel('Pixel Intensity', fontsize=10)
    ax8.set_ylabel('Density', fontsize=10)
    ax8.set_title('Intensity Distribution', fontsize=12, fontweight='bold')
    ax8.legend()
    ax8.grid(alpha=0.3)
    
    # Row 3: PSF and additional analysis (if PSF provided)
    if psf is not None:
        ax9 = fig.add_subplot(gs[2, 0])
        ax9.imshow(psf, cmap='gray')
        ax9.set_title("Point Spread Function (PSF)", fontsize=12, fontweight='bold')
        ax9.axis('off')
        
        # Cross-section comparison
        ax10 = fig.add_subplot(gs[2, 1:3])
        mid_row = original.shape[0] // 2
        ax10.plot(orig_norm[mid_row, :], label='Original', linewidth=2, alpha=0.8)
        ax10.plot(blur_norm[mid_row, :], label='Blurry', linewidth=2, alpha=0.8)
        ax10.plot(deconv_norm[mid_row, :], label='Deconvolved', linewidth=2, alpha=0.8)
        ax10.set_xlabel('Pixel Position', fontsize=10)
        ax10.set_ylabel('Intensity', fontsize=10)
        ax10.set_title(f'Cross-section (Row {mid_row})', fontsize=12, fontweight='bold')
        ax10.legend()
        ax10.grid(alpha=0.3)
        
        # Summary text
        ax11 = fig.add_subplot(gs[2, 3])
        ax11.axis('off')
        summary_text = f"""
PERFORMANCE SUMMARY
{'='*30}

PSNR Gain: {metrics['psnr_gain']:.2f} dB
SSIM Gain: {metrics['ssim_gain']:.4f}
MSE Reduction: {metrics['mse_reduction_pct']:.1f}%

Sharpness Recovery:
{metrics['sharpness_recovery_pct']:.1f}% of original

Edge Recovery:
{metrics['edge_recovery_pct']:.1f}% of original

{'='*30}
        """
        ax11.text(0.1, 0.5, summary_text, fontsize=10, family='monospace',
                 verticalalignment='center', bbox=dict(boxstyle='round', 
                 facecolor='wheat', alpha=0.5))
    
    plt.suptitle('Richardson-Lucy Deconvolution Performance Analysis', 
                 fontsize=16, fontweight='bold', y=0.98)
    plt.show()


# Usage with your variables:
# metrics = comprehensive_deconvolution_analysis(original_sharp, blurry_bs, rl, psf)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject